In [ ]:
%pip install qiskit==1.2.4
%pip install qiskit-aer==0.15.1
%pip install pylatexenc==2.10

In [1]:
from qiskit import QuantumCircuit
from qiskit.converters import circuit_to_gate
from qiskit.visualization import array_to_latex
from qiskit.quantum_info import Operator
from qiskit.quantum_info import Statevector
from qiskit import transpile 
from qiskit.providers.basic_provider import BasicSimulator
from qiskit.visualization import plot_histogram
from qiskit.circuit import ControlledGate
import math 


# The aim of the assignment is to simulate the Ekert91 key distribution protocol.

# This notebook is for a simulation of the protocol with an attacker, to demonstrate that the attacker can be detected.

#student ID 2762449c

#create two qubits expressed by the needed Bell state and entangle them
def entangledPair():
    q = QuantumCircuit(2,2) 
    q.h(0)
    q.cx(0,1)
    q.x(1)
    q.z(0)

    return q


#definitions

a = 1/math.sqrt(3)
b = math.sqrt(2)/math.sqrt(3)

n = 10

#gate definitions for X, Z, V, W
    
x = QuantumCircuit(1)
x.h(0)
X = circuit_to_gate(x)


z = QuantumCircuit(1) #qiskit already measures in Z
Z = circuit_to_gate(z)

w = QuantumCircuit(1)
w.ry(-math.pi/4, 0)
W = circuit_to_gate(w) 

v = QuantumCircuit(1) 
v.ry(math.pi/4, 0)
V = circuit_to_gate(v)



#so 1/3 probability for a, 2/3 for b. if b is chosen, use another qubit with
#equal probabilities for both states and decide from there which basis to use
basis_finder_1 = QuantumCircuit(1,1)
basis_finder_1.initialize([a,b],0)
basis_finder_1.measure(0,0)

basis_finder_2 = QuantumCircuit(1,1)
basis_finder_2.h(0)
basis_finder_2.measure(0,0)

backend = BasicSimulator()

compiled = transpile(basis_finder_1, backend)
compiled_2 = transpile(basis_finder_2,backend)

        
def find_choices():
    choices=[]
    for _ in range(math.ceil(9*n/2)):
        job = backend.run(compiled,shots=1)
        res=job.result()
        counts=res.get_counts(compiled)
        choices.append(int(max(counts, key=counts.get)))


    for i in range(len(choices)):
        if choices[i]==1:
            job = backend.run(compiled_2,shots=1)
            res=job.result()
            counts=res.get_counts(compiled_2)
            choices[i] = int(max(counts, key=counts.get))
            choices[i]+=1

    return choices

def average(c,n,d): 
    backend = BasicSimulator()
    compiled = transpile(c, backend)
    job_sim = backend.run(compiled, shots=n)
    result_sim = job_sim.result() 
    counts = result_sim.get_counts(compiled)

    count00 = counts.get("00",0) 
    d["00"]+=count00
    
    count01 = counts.get("01",0) 
    d["01"]-=count01
    
    count10 = counts.get("10",0) 
    d["10"]-=count10
    
    count11 = counts.get("11",0) 
    d["11"]+=count11
    bitstring = max(counts, key=counts.get)
   
    alice_bit = int(bitstring[-1])
    bob_bit   = int(bitstring[0])


    return d, alice_bit, bob_bit

def eves_basis():
    job = backend.run(compiled_2,shots=1)
    res=job.result()
    counts=res.get_counts(compiled_2)
    choice = int(max(counts, key=counts.get))
    return choice
    
def eavesdrop(qc,i):
    basis = eves_basis()
    if basis == 0:
        qc.append(W,[i])
    else:
        qc.append(Z,[i])
    qc.measure(i,i)
    qc.reset(i)
    if basis==0: 
        qc.append(W,[i])
    else:
        qc.append(Z,[i])
    qc.x(i).c_if(i,1)
    return qc
    
def create_circuit(alice_choice,bob_choice):
    qc = entangledPair()
    
    qc = eavesdrop(qc, 1)
    
    if alice_choice==0:
        qc.append(X,[0])
    elif alice_choice==1:
        qc.append(W,[0])
    elif alice_choice==2:
        qc.append(Z,[0])

    if bob_choice==0:
        qc.append(W,[1])
    elif bob_choice==1:
        qc.append(Z,[1])
    else:
        qc.append(V,[1])
    qc.measure([0,1],[0,1])

    
    return qc
    

def key_creation_for_alice(alices_choices,bobs_choices,alices_results):
    key=""
    remainder_choices=[]
    remainder_results=[]
    for i in range(len(alices_choices)):
        if alices_choices[i]==1 and bobs_choices[i]==0:
            key+=str(alices_results[i])
  
        elif alices_choices[i]==2 and bobs_choices[i]==1:
            key+=str(alices_results[i])


    return key
            
def key_creation_for_bob(alices_choices,bobs_choices,bobs_results):
    key=""
    remainder_choices=[]
    remainder_results=[]
    for i in range(len(bobs_choices)):
        if alices_choices[i]==1 and bobs_choices[i]==0:
            key+=str(1-bobs_results[i])

        elif alices_choices[i]==2 and bobs_choices[i]==1:
            key+=str(1-bobs_results[i])


    return key 

def convert_bit(bit):
    return 1 if bit==0 else -1
            
def div(num,dem):
    return num/dem if dem!=0 else 0


def get_key_length_n():
    alices_choices=find_choices()
    bobs_choices = find_choices()

    avgs = []
    N=1000
    alices_results = []
    bobs_results = []

    A1B1 = {"00":0,"01":0,"10":0,"11":0}
    A1B3 = {"00":0,"01":0,"10":0,"11":0}
    A3B1 = {"00":0,"01":0,"10":0,"11":0}
    A3B3 = {"00":0,"01":0,"10":0,"11":0}
    dud = {"00":0,"01":0,"10":0,"11":0}
    count_A1B1, count_A1B3,count_A3B1,count_A3B3 = 0,0,0,0
    
    for i in range(math.ceil(9*n/2)):
        
        qc = create_circuit(alices_choices[i],bobs_choices[i])
        

        if alices_choices[i]==1:
            dud,alice_bit,bob_bit=average(qc,N,dud)
            
        if alices_choices[i]==0:
            if bobs_choices[i]==0:
                A1B1,alice_bit,bob_bit = average(qc,N,A1B1)
                count_A1B1+=N
 
            elif bobs_choices[i]==2:
                A1B3,alice_bit,bob_bit = average(qc,N,A1B3)
                count_A1B3+=N
            else:
                dud,alice_bit,bob_bit = average(qc,N,dud)
    
                
        if alices_choices[i]==2 :
            if bobs_choices[i]==0:
         
                A3B1,alice_bit,bob_bit = average(qc,N,A3B1)
                count_A3B1+=N
         
            elif bobs_choices[i]==2:       
                A3B3,alice_bit,bob_bit = average(qc,N,A3B3)
                count_A3B3+=N

            else:
                dud,alice_bit,bob_bit = average(qc,N,dud)
        
        alices_results.append(alice_bit)
        bobs_results.append(bob_bit)
 
    avg_A1B1=div(A1B1["00"]+A1B1["01"]+A1B1["10"]+A1B1["11"],count_A1B1)
    avg_A1B3=div(A1B3["00"]+A1B3["01"]+A1B3["10"]+A1B3["11"],count_A1B3)
    avg_A3B1=div(A3B1["00"]+A3B1["01"]+A3B1["10"]+A3B1["11"],count_A3B1)
    avg_A3B3=div(A3B3["00"]+A3B3["01"]+A3B3["10"]+A3B3["11"],count_A3B3)
    
    S = abs(avg_A1B1 - avg_A1B3 + avg_A3B1 + avg_A3B3)
    alices_key= key_creation_for_alice(alices_choices,bobs_choices,alices_results)
    bobs_key= key_creation_for_bob(alices_choices,bobs_choices,bobs_results)
    return alices_key,bobs_key,S
    

alices_key,bobs_key,S = "","",0


while len(alices_key)!=n or len(bobs_key)!=n:
    alices_key,bobs_key,S=get_key_length_n()
 

print(alices_key,bobs_key,S)
if S<=2:
    print("Only classical correlation")
elif S<=(2.7):
    print("Attacker possible")
else:
    print("Successful")

ModuleNotFoundError: No module named 'qiskit'